In [1]:
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from scipy import sparse

In [2]:
column = "word_seg"
train = pd.read_csv('../input/new_data/train_set.csv')
test = pd.read_csv('../input/new_data/test_set.csv')
test_id = test["id"].copy()
y_all = (pd.Series(train['class'])-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


### tfidf提取特征

In [3]:
vec_doc = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_doc = vec_doc.fit_transform(train[column])
test_term_doc = vec_doc.transform(test[column])

In [4]:
# sparse.save_npz('../feature/text_tifi/trn_term_doc.npz', trn_term_doc)
# sparse.save_npz('../feature/text_tifi/test_term_doc.npz', test_term_doc)

# trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
# test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')

In [5]:
vec_word = TfidfVectorizer(ngram_range=(1,1),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_word = vec_word.fit_transform(train[column])
test_term_word = vec_word.transform(test[column])

In [6]:
# sparse.save_npz('../feature/text_tifi/trn_term_word.npz', trn_term_word)
# sparse.save_npz('../feature/text_tifi/test_term_word.npz', test_term_word)

# trn_term_word = sparse.load_npz('../feature/text_tifi/trn_term_word.npz')
# test_term_word = sparse.load_npz('../feature/text_tifi/test_term_word.npz')

In [7]:
vec_word_nramThree = TfidfVectorizer(ngram_range=(3,3),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_word_three = vec_word_nramThree.fit_transform(train[column])
test_term_word_three = vec_word_nramThree.transform(test[column])

In [8]:
# sparse.save_npz('../feature/text_tifi/trn_term_word_three.npz', trn_term_word_three)
# sparse.save_npz('../feature/text_tifi/test_term_word_three.npz', test_term_word_three)

# trn_term_word_three = sparse.load_npz('../feature/text_tifi/trn_term_word_three.npz')
# test_term_word_three = sparse.load_npz('../feature/text_tifi/test_term_word_three.npz')

In [9]:
train_feaure =  hstack([trn_term_doc,trn_term_word,trn_term_word_three]).tocsr()
del trn_term_doc,trn_term_word,trn_term_word_three
import gc
gc.collect()

45

In [10]:
vec_word_nramfour = TfidfVectorizer(ngram_range=(4,4),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='word')
trn_term_word_four = vec_word_nramfour.fit_transform(train[column])
test_term_word_four = vec_word_nramfour.transform(test[column])

In [11]:
# sparse.save_npz('../feature/text_tifi/trn_term_word_four.npz', trn_term_word_four)
# sparse.save_npz('../feature/text_tifi/test_term_word_four.npz', test_term_word_four)

# trn_term_word_four = sparse.load_npz('../feature/text_tifi/trn_term_word_four.npz')
# test_term_word_four = sparse.load_npz('../feature/text_tifi/test_term_word_four.npz')

In [12]:
vec_word_nramfive = TfidfVectorizer(ngram_range=(5,5),min_df=3, max_df=1.0,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=400000,analyzer='char')
trn_term_word_five = vec_word_nramfive.fit_transform(train[column])
test_term_word_five = vec_word_nramfive.transform(test[column])

In [13]:
# sparse.save_npz('../feature/text_tifi/trn_term_word_five.npz', trn_term_word_five)
# sparse.save_npz('../feature/text_tifi/test_term_word_five.npz', test_term_word_five)
# trn_term_word_five = sparse.load_npz('../feature/text_tifi/trn_term_word_five.npz')
# test_term_word_five = sparse.load_npz('../feature/text_tifi/test_term_word_five.npz')

In [14]:
vec_char = TfidfVectorizer(ngram_range=(1,2),min_df=3,use_idf=1,smooth_idf=1, sublinear_tf=1,max_features=10000,analyzer='char')
trn_char = vec_char.fit_transform(train["article"])
test_char = vec_char.transform(test["article"])

In [15]:
# sparse.save_npz('../feature/text_tifi/trn_char.npz', trn_char)
# sparse.save_npz('../feature/text_tifi/test_char.npz', test_char)

# trn_char = sparse.load_npz('../feature/text_tifi/trn_char.npz')
# test_char = sparse.load_npz('../feature/text_tifi/test_char.npz')

In [16]:
docLen  =pd.DataFrame( np.array(train[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
doclen_word = pd.DataFrame( np.array(train['article'].map(lambda x : len(x.split(" ")))/train[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))
docLen_test  =pd.DataFrame( np.array(test[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
doclen_word_test = pd.DataFrame( np.array(test['article'].map(lambda x : len(x.split(" ")))/test[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))

In [17]:
import gc
del train, test
gc.collect()

14

In [18]:

train_feaure =  hstack([train_feaure, trn_term_word_four,trn_char,doclen_word,trn_term_word_five]).tocsr()
del trn_term_word_four,trn_char,doclen_word,trn_term_word_five
gc.collect()

0

In [19]:
# test_festure.shape

### 使用计数后tfidf,效果提升不大，维度上升60w,试过chi2降维，效果降低。，10折比5折有用

In [20]:
train_feaure.shape

(102277, 1638740)

In [21]:
# def feature_select(corpus, labels, k=1000):
#     """
#     select top k features through chi-square test
#     """
#     bin_cv = CountVectorizer(binary=True)
#     le = LabelEncoder()
#     X = bin_cv.fit_transform(corpus)
#     print("countVector Shap: ",X.shape)
#     y = le.fit_transform(labels).reshape(-1, 1)

#     k = min(X.shape[1], k)
#     skb = SelectKBest(chi2, k=k)
#     skb.fit(X, y)

#     feature_ids = skb.get_support(indices=True)
#     feature_names = bin_cv.get_feature_names()
#     vocab = {}

#     for new_fid, old_fid in enumerate(feature_ids):
#         feature_name = feature_names[old_fid]
#         vocab[feature_name] = new_fid

#     # we only care about the final extracted feature vocabulary
#     return vocab

### 降维

In [22]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler(with_mean=False)
# ss.fit(train_feaure)
# X_train_std = ss.transform(train_feaure)


In [23]:
# skb = SelectKBest(chi2, k=800000)
# train_feaure_chi = skb.fit_transform(X_train_std, y_all)

In [24]:
# del train_feaure
# gc.collect()

In [25]:
# N = 2
# skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=42)
# xx_cv = []
# xx_pred = []
# for train_in,test_in in skf.split(train_feaure,y_all):
#     X_train,X_test,y_train,y_test =train_feaure[train_in],train_feaure[test_in],y_all[train_in],y_all[test_in]
#     lin_svm = svm.LinearSVC(C=1.0,random_state=2018,max_iter=5000)
#     lin_svm.fit(X_train,y_train)
#     print('Start predicting...')
#     preds = lin_svm.predict(X_test)
# #     y_pred = pd.Series(y_pred).map(lambda x:x+1)
#     fscoer = f1_score(y_test, preds, average='micro')
#     print(fscoer)
#     xx_cv.append(fscoer)
# #     xx_pred.append(lin_svm.predict(test_term_doc))
#     print(classification_report(y_test,preds))
#     print('-------------------end---------------------------')
# print('END:  ',np.mean(xx_cv))

In [26]:
# X_train,X_test, y_train, y_test =train_test_split(train_feaure,y_all,test_size=0.3, random_state=0)

In [27]:
# for i in [0.1,0.4,0.6,0.7,0.8,0.9]:
#     lin_svm = svm.LinearSVC(C=i,random_state=2018,max_iter=500)
#     lin_svm.fit(X_train,y_train)
#     print('Start predicting...     ',i)
#     preds = lin_svm.predict(X_test)
#     fscoer = f1_score(y_test, preds, average='micro')
#     print(fscoer)
#     print(classification_report(y_test,preds))

In [28]:
lr = LogisticRegression(C=6,dual=True)
lr.fit(train_feaure,y_all)
# ypred = lr.predict_proba(X_test)
# preds=np.argmax(ypred,axis=1)
# fscoer = f1_score(y_test, preds, average='micro')
# print(fscoer)
# print(classification_report(y_test,preds))
#0.7882


LogisticRegression(C=6, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
del train_feaure
gc.collect()

0

In [30]:
test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
test_term_word = sparse.load_npz('../feature/text_tifi/test_term_word.npz')
test_term_word_three = sparse.load_npz('../feature/text_tifi/test_term_word_three.npz')
test_term_word_four = sparse.load_npz('../feature/text_tifi/test_term_word_four.npz')
test_term_word_five = sparse.load_npz('../feature/text_tifi/test_term_word_five.npz')
test_char = sparse.load_npz('../feature/text_tifi/test_char.npz')
test_festure =  hstack([test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test,test_term_word_five]).tocsr()
del test_term_doc,test_term_word,test_term_word_three,test_term_word_four,test_char,doclen_word_test,test_term_word_five
gc.collect()


90

In [31]:
ypred = lr.predict_proba(test_festure)
preds=np.argmax(ypred,axis=1)
# fscoer = f1_score(y_test, preds, average='micro')
# print(fscoer)
# print(classification_report(y_test,preds))

In [33]:
# test_prob=pd.DataFrame(ypred)
# test_prob.columns=["class_prob_%s"%i for i in range(1,20)]
test_prob["id"]=list(test_id)
test_prob.to_csv('../pro/prob_lr_baseline3.csv',index=None)

In [ ]:
res = pd.DataFrame()
res['id'] = test_id
res['class'] = pd.Series(preds).map(lambda x : x+1)
print(res.head())
res.to_csv('../data/baseline_lr_ngre5.csv',header=True,index=False)

In [ ]:

# from sklearn.model_selection import StratifiedKFold
# N = 2
# from sklearn.preprocessing import OneHotEncoder
# skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=42)
# from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
# xx_cv = []
# xx_pred = []
# for train_in,test_in in skf.split(trn_term_doc,y_all):
#     X_train,X_test,y_train,y_test =trn_term_doc[train_in],trn_term_doc[test_in],y_all[train_in],y_all[test_in]
#     xgbc = XGBClassifier(
#         learning_rate =0.1,
#         n_estimators=500,
#         max_depth=6,
#         min_child_weight=0,
#         gamma=0,
#         subsample=0.7,
#         colsample_bytree=0.7,
#         objective= 'multi:softmax',
#         num_class=19,
#         eval_metric='merror',
#         nthread=-1,
#         scale_pos_weight=1,
#         seed=27)
#     print('Start predicting...')
#     xgbc_enc = OneHotEncoder()  
#     lin_svm = svm.LinearSVC(C=1.0,random_state=2018,max_iter=5000)
#     xgbc.fit(X_train,y_train)
#     xgbc_enc.fit(xgbc.apply(X_train)[:, :])
#     lin_svm.fit(xgbc_enc.transform(xgbc.apply(X_train)[:, :]), y_train)
#     y_xgbc_lr_test = lin_svm.predict(xgbc_enc.transform(xgbc.apply(X_test)[:,:]))
# #     preds=np.argmax(y_xgbc_lr_test,axis=1)
#     print(y_xgbc_lr_test)
# #     y_pred = pd.Series(y_pred).map(lambda x:x+1)
#     fscoer = f1_score(y_test, y_xgbc_lr_test, average='micro')
#     print(fscoer)
#     xx_cv.append(fscoer)
# #     xx_pred.append(lin_svm.predict(test_term_doc))
#     print(classification_report(y_test,y_xgbc_lr_test))
#     print('-------------------end---------------------------')
# print('END:  ',np.mean(xx_cv))

In [34]:
test_prob

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19,id
0,0.001389,0.000817,0.001376,0.004756,0.970469,0.000833,0.002470,0.000395,0.001814,0.000295,0.001118,0.000638,0.000951,0.007957,0.001413,2.314474e-04,0.000507,0.000701,0.001870,0
1,0.017763,0.001139,0.002171,0.934325,0.000744,0.001718,0.001016,0.000264,0.001037,0.000981,0.000294,0.010586,0.000377,0.001217,0.000883,3.027055e-04,0.000267,0.023374,0.001541,1
2,0.002366,0.001450,0.087272,0.002214,0.006902,0.000404,0.006886,0.161686,0.004028,0.000489,0.001526,0.006825,0.690399,0.000389,0.012098,5.041433e-07,0.008141,0.001253,0.005672,2
3,0.000627,0.000141,0.001263,0.984402,0.000487,0.007826,0.000339,0.001875,0.000282,0.000111,0.000682,0.000404,0.000118,0.000385,0.000254,6.648732e-05,0.000299,0.000241,0.000199,3
4,0.020255,0.000219,0.000282,0.000614,0.939391,0.000792,0.013479,0.000871,0.001239,0.003014,0.004459,0.000510,0.001357,0.005976,0.002398,2.562064e-05,0.000673,0.000409,0.004035,4
5,0.008663,0.002712,0.002545,0.003496,0.894956,0.005669,0.000567,0.014711,0.002147,0.008816,0.033556,0.002717,0.003374,0.005299,0.003906,7.434454e-04,0.000615,0.003057,0.002450,5
6,0.002166,0.000732,0.009548,0.001909,0.000222,0.000553,0.001196,0.000322,0.000228,0.016946,0.000053,0.002270,0.000221,0.001590,0.940207,1.096721e-03,0.002427,0.003950,0.014362,6
7,0.011475,0.011712,0.003332,0.002313,0.001111,0.002506,0.005542,0.002429,0.001344,0.013812,0.002802,0.005106,0.007772,0.001111,0.003720,5.204894e-04,0.107628,0.002668,0.813098,7
8,0.002529,0.000261,0.965832,0.001175,0.000302,0.001298,0.000171,0.000437,0.009441,0.000632,0.004642,0.002304,0.003677,0.001473,0.000941,1.688997e-05,0.001874,0.001759,0.001236,8
9,0.049471,0.002321,0.001380,0.004014,0.001173,0.010597,0.006884,0.169590,0.020576,0.031910,0.006740,0.552579,0.005228,0.001972,0.000330,2.129438e-03,0.000370,0.060689,0.072046,9


In [ ]:
# from sklearn.model_selection import StratifiedKFold
# N = 3
# skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=42)
# from sklearn.metrics import roc_auc_score

# xx_cv = []
# xx_pred = []
# for train_in,test_in in skf.split(xtrain_scl,y_all):
#     X_train,X_test,y_train,y_test = xtrain_scl[train_in],xtrain_scl[test_in],y_all[train_in],y_all[test_in]
#     clf = MultinomialNB(alpha = 0.01)
#     clf.fit(X_train, y_train);
#     print('Start predicting...')
#     preds = clf.predict(X_test)
# #     y_pred = pd.Series(y_pred).map(lambda x:x+1)
#     fscoer = f1_score(y_test, preds, average='micro')
#     print(fscoer)
#     xx_cv.append(fscoer)
# #     xx_pred.append(lin_svm.predict(test_term_doc))
#     print(classification_report(y_test,preds))
#     print('-------------------end---------------------------')
# print('END:  ',np.mean(xx_cv))